MAC 5768 - Visão e Processamento de Imagens

Segundo Semestre - 2020

Equipe:

    Ciro B Rosa - ciro.rosa@alumni.usp.br
    Josilton Sousa - josilton.sousa@gmail.com

Projeto de elaboração de tarefas do curso de Visão e Processamento de Imagens.


Fase 2 - Parte 1 - Entrega em 16/11/2020

Objetivos:
* A partir de 1080 fotos coloridas de 10 categorias de objetos (originalDataset), criar duplicatas em escala de cinza e armazená-las na pasta originalGrayDataset.
* A partir das imagens em escala cinza, obter um dataset aumentado e armazenar em augmentedDataset.

As funções para geração do augmentedDataset são:
* Escala de cinza
* Soma do fundo da imagem (gradiente) com a imagem cinza
* Log2 da imagem cinza
* Função Gama (exponencial da imagem)
* Média e convolução, com o uso de janela uniforme 3x3.


Código para geração do originalGrayDataset a partir do originalDataset

Este código terá as seguintes funções:
* Leitura dos metadados originais
* Geração de imagens em escala cinza
* Criação de metadados específicos para as imagens geradas
* Gravação de dados em originalGrayDataset

In [1]:
import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
import pandas as pd

# arquivo de metadados em cores
pasta1 = "./originalDataset/"
metafile1 = "grade.csv"
filename1 = pasta1 + metafile1
df = pd.read_csv(filename1, sep=";")
print(df.head(5), "\n")

# arquivo de metadados em cinza
pasta2 = "./originalGrayDataset/"
metafile2 = "grade.csv"
filename2 = pasta2 + metafile2

# adequação do nome de arquivo nos metadados
def fix_jpeg_name(arq):
    arq2 = int(arq[:-4])
    arq2 = str(arq2) + ".jpg"
    return arq2
    

# quantidade de fotos
n = len(df.arquivo)

# criar dataframe vazio para metadados em cinza
df2 = pd.DataFrame(data=None, columns=df.columns)
df2.insert(loc=5, column="transformacao", value=[])
df2_length = len(df2)

# transformação de fotos cinza
for ind in df.index:
    #print(df.loc[ind])

    # ler foto colorida para a lista
    arq1 = df["arquivo"][ind]
    arq1 = fix_jpeg_name(arq1)
    f1 = pasta1 + arq1
    img = io.imread(f1)
    
    # gerar foto cinza
    img_gray = rgb2gray(img)
    img_gray2 = img_as_ubyte(img_gray)      # muda a representação de [-1, 1] para [0, 255]
    #img_gray3 = img_gray.astype(np.uint8)   # "arredonda" para inteiro entre [0, 255]
         
    # nome do arquivo da foto cinza
    arq1 = int(arq1[:-4])
    arq2 = arq1 + n
    arq2 = str(arq2) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto cinza
    io.imsave(f, img_gray2)
     
    # gravar dados da foto cinza no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "cinza", df.responsavel[ind], arq2]
    df2.loc[df2_length] = l
    df2_length += 1
    
    # roda o loop apenas uma vez
    #break

# salvar metadados em cinza
print(df2.head(5), "\n")
df2.to_csv(filename2, index = False, header=True, sep=";")


   sequencia objeto tipo_obj   fundo  iluminacao responsavel arquivo
0          1  garfo        a  branco  indoor dia    Josilton  1 .jpg
1          2  garfo        a  branco  indoor dia    Josilton  2 .jpg
2          3  garfo        a  branco  indoor dia    Josilton  3 .jpg
3          1   faca        a  branco  indoor dia    Josilton  4 .jpg
4          2   faca        a  branco  indoor dia    Josilton  5 .jpg 

  sequencia objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0         1  garfo        a  branco  indoor dia         cinza    Josilton   
1         2  garfo        a  branco  indoor dia         cinza    Josilton   
2         3  garfo        a  branco  indoor dia         cinza    Josilton   
3         1   faca        a  branco  indoor dia         cinza    Josilton   
4         2   faca        a  branco  indoor dia         cinza    Josilton   

    arquivo  
0  1081.jpg  
1  1082.jpg  
2  1083.jpg  
3  1084.jpg  
4  1085.jpg   



Código para geração do augmentedDataset a partir do originalGrayDataset:

* Copia-se a manualmente a base de dados originalGrayDataset para augmentedDataset;
* Utiliza-se o código a seguir para complementar a base augmentedDataset com as demais transformações: gradiente, log2, gama/exponenciação, convolução;
* Cria-se arquivo com os metadados correspondentes.


In [2]:
import numpy as np
from skimage import io
from scipy.signal import convolve2d as conv2
import pandas as pd

# definição de funções de transformação
def transf_grad(img):
    img_aux = np.gradient(img)
    img2 = np.sqrt(np.square(img_aux[0]) + np.square(img_aux[1]))
    return img2

def transf_log2(img, c):
    img2 = c * np.log2(1.0 + img)
    return img2

def transf_gama(img, c, gama):
    img2 = c * np.power(img, gama)
    return img2

# adequação do nome de arquivo nos metadados
def fix_jpeg_name(arq):
    arq2 = int(arq[:-4])
    arq2 = str(arq2) + ".jpg"
    return arq2


# arquivo de metadados em cinza
pasta1 = "./originalGrayDataset/"
metafile1 = "grade.csv"
filename1 = pasta1 + metafile1
df = pd.read_csv(filename1, sep=";")
print(df.head(5), "\n")

# arquivo de metadados aumentados
pasta2 = "./augmentedDataset/"
metafile2 = "grade.csv"
filename2 = pasta2 + metafile2

# criar dataframe para metadados aumentados
df2 = pd.DataFrame(data=df, index=df.index, columns=df.columns)

# número do último arquivo
n = df2.tail(1)
n = n["arquivo"]
n = n.values[0]
n = str(n[:-4])
n = int(n)


# transformação de fotos cinza
for ind in df.index:
    #print(df.loc[ind])

    # ler foto cinza
    arq1 = df["arquivo"][ind]
    arq1 = fix_jpeg_name(arq1)
    f1 = pasta1 + arq1
    img_gray = io.imread(f1)
    
    
    ### soma de fundo (gradiente) com foto cinza
    ###
    img_grad = transf_grad(img_gray)
    img_grad = img_gray + img_grad
    img_grad = img_grad.astype(np.uint8)
     
    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_grad)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "grad", df.responsavel[ind], arq2]
    df2.loc[n] = l
    
    ### foto log2
    ###
    img_log2 = transf_log2(img_gray, c = 1)
    img_log2 = img_log2.astype(np.uint8)
    
    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_log2)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "log", df.responsavel[ind], arq2]
    df2.loc[n] = l
    
    ### foto exponencial
    ###
    img_gama = transf_gama(img_gray, c = 1, gama = 2)
    img_gama = img_gama.astype(np.uint8)

    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_gama)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "exp", df.responsavel[ind], arq2]
    df2.loc[n] = l

    ### média e convolução
    ###
    psf = np.ones((3, 3)) / 9
    img_conv = conv2(img_gray, psf, 'same')
    img_conv = img_conv.astype(np.uint8)

    # nome de arquivo da foto gerada
    n += 1
    arq2 = str(n) + ".jpg"
    f = pasta2 + arq2
    
    # salvar foto gerada
    io.imsave(f, img_conv)
     
    # gravar dados da foto gerada no dataframe
    l = [df.sequencia[ind], df.objeto[ind], df.tipo_obj[ind], df.fundo[ind], df.iluminacao[ind], "conv", df.responsavel[ind], arq2]
    df2.loc[n] = l
    
    # roda o loop apenas uma vez
    #break

# salvar metadados aumentados
print(df2.head(5), "\n")
df2.to_csv(filename2, index = False, header=True, sep=";")


   sequencia objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0          1  garfo        a  branco  indoor dia         cinza    Josilton   
1          2  garfo        a  branco  indoor dia         cinza    Josilton   
2          3  garfo        a  branco  indoor dia         cinza    Josilton   
3          1   faca        a  branco  indoor dia         cinza    Josilton   
4          2   faca        a  branco  indoor dia         cinza    Josilton   

    arquivo  
0  1081.jpg  
1  1082.jpg  
2  1083.jpg  
3  1084.jpg  
4  1085.jpg   



<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2162.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2166.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2170.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2174.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2178.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2182.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2186.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2190.jpg is a low contrast image
  io.imsave(f, img_log2)


<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2426.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2430.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2434.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2438.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2442.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2446.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2450.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2454.jpg is a low contrast image
  io.imsave(f, img_log2)


<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2690.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2694.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2698.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2702.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2706.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2710.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2714.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2718.jpg is a low contrast image
  io.imsave(f, img_log2)


<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2954.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2958.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2962.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2966.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2970.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2974.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2978.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/2982.jpg is a low contrast image
  io.imsave(f, img_log2)


...

<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5594.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5598.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5602.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5606.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5610.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5614.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5618.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5622.jpg is a low contrast image
  io.imsave(f, img_log2)


<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5858.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5862.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5866.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5870.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5874.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5878.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5882.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/5886.jpg is a low contrast image
  io.imsave(f, img_log2)


<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6122.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6126.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6130.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6134.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6138.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6142.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6146.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6150.jpg is a low contrast image
  io.imsave(f, img_log2)


<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6386.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6390.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6394.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6398.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6402.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6406.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6410.jpg is a low contrast image
  io.imsave(f, img_log2)
<ipython-input-2-eba048850380>:90: UserWarning: ./augmentedDataset/6414.jpg is a low contrast image
  io.imsave(f, img_log2)


   sequencia objeto tipo_obj   fundo  iluminacao transformacao responsavel  \
0          1  garfo        a  branco  indoor dia         cinza    Josilton   
1          2  garfo        a  branco  indoor dia         cinza    Josilton   
2          3  garfo        a  branco  indoor dia         cinza    Josilton   
3          1   faca        a  branco  indoor dia         cinza    Josilton   
4          2   faca        a  branco  indoor dia         cinza    Josilton   

    arquivo  
0  1081.jpg  
1  1082.jpg  
2  1083.jpg  
3  1084.jpg  
4  1085.jpg   

